In [1]:
import open3d as o3d
import laspy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#used to load .las data into a laspy object.
def load_laspy(las_location):
      input_las = laspy.read(las_location)
      
      return input_las

#used to return a list of unique classifications present in the input LAS file.
def get_list_classifications(input_las):
    classifications = list(set(input_las.classification))
    classifications.sort()
    
    return classifications

#Used to return a Python dictionary of laspy objects of the entire dataset as well as individual classifications.
def get_classifications_laspy(input_las):
    #get a unique list of the classifications currently contained in the dataset.
    classifications = get_list_classifications(input_las=input_las)
    #create dictionary with complete data for each province
    d = {"original_dataset": input_las}
    print("The classifications contained in this .las dataset are: ")
    print(classifications)
    print("Creating a python dictionary containing the entire dataset and individual classifications as separate laspy objects.")
    for classification in classifications:
        print("Saving classification number: " +str(classification))
        classified_points = laspy.create(point_format=input_las.header.point_format, file_version=input_las.header.version)
        classified_points.points = input_las.points[input_las.classification == classification]
        
        d["classification_{}".format(str(classification))] = classified_points
    
    return d

#Optional function which takes the dictionary of laspy objects and converts every classification
#   into individual .LAS files 
def laspy_classifications_to_las(classifications_laspy):
    for classification_las in classifications_laspy:
        if classification_las == "original_dataset":
            pass
            print("skipped exporting the original dataset.")
        else:
            print("processing {} to a LAS file: ".format(classification_las))
            print(classifications_laspy[classification_las])
            classifications_laspy[classification_las].write("classification_{}.las".format(classification_las.split("_")[1]))
            print("done")

# Convert a .LAS file into a pandas object.
def convert_laspy_pandas(input_las):
    # Convert data into pandas DataFrame
    df = pd.DataFrame({"X":input_las.X,"Y":input_las.Y,"Z":input_las.Z,
      "x":np.array(input_las.x),"y":np.array(input_las.y),"z":np.array(input_las.z),
     'intensity': input_las.intensity,
      'classification': input_las.classification,
      'return_number': np.array(input_las.return_number),
      'number_of_returns':np.array(input_las.number_of_returns),
      'synthetic':np.array(input_las.synthetic),
      'key_point':np.array(input_las.key_point),
      'withheld':np.array(input_las.withheld),
      'overlap':np.array(input_las.overlap),
      'scanner_channel':np.array(input_las.scanner_channel),
      'scan_direction_flag':np.array(input_las.scan_direction_flag),
      'user_data':input_las.user_data,
      'scan_angle':input_las.scan_angle,
      'point_source_id':input_las.point_source_id,
      'gps_time':input_las.gps_time    
      })

    return df

def get_selected_classes(classifications,input_las):
    classes = get_list_classifications(input_las=input_las)
    available_classes = classes
    available_classes.append(1000) #to represent the entire dataset
    class_selections = []
    current_selection = -10

    while current_selection == -10:
        print("Available classifications: " + str(available_classes))
        try:
            current_selection = input("Enter a number from the list of available classifications. To get the full dataset enter 1000 : \n")
            if current_selection.isnumeric() == False:
                print("Invalid Character Input.")
                raise ValueError()
            current_selection = int(current_selection)
            if current_selection not in available_classes:
                print("The classification number you selected: {}, is not an available class item.".format(current_selection))
                cancel = input("Would you like to exit instead? 'Y/N' ").upper() + "Y"
                if cancel[0] == "Y":
                    current_selection = -1000
                    print("Exiting and returning empty classification selection list.")
                    class_selections = []    
                else: 
                    raise ValueError()
            if (current_selection != -1000) & (current_selection != 1000):

                if 1000 in available_classes:
                    available_classes.remove(1000)
                class_selections.append(current_selection)
                print("Class {} added to dataframe.".format(str(current_selection)))
                available_classes.remove(current_selection)
                ask = input("Would you like to add another classification to the dataframe? 'Y/N' ").upper() + "Y"

                #If user prompts to add another classification, reset variable and start again.
                if ask[0] == "Y":
                    current_selection = -10
                #If user prompts to stop, exit loop and create pandas dataframe.
                else:
                    print("Classifications have been selected.")
                    class_selections.sort()

            elif current_selection == 1000:
                print("The entire dataset has been selected")
                class_selections = get_list_classifications(input_las=input_las)
                print("All classes have been added to the dataframe.")
                class_selections.sort()

        except ValueError:
            print("Please select an input from the available classification list. \n")
            current_selection = -10

    
    return class_selections

def create_combined_classifications_dataframe(class_selections,df):
    if len(class_selections) > 0:
        #initialize the dataframe with the first set of classifications
        combined_df = df.loc[df["classification"] == class_selections[0]]
        #iterate and append remaining classifications 
        for selection in class_selections[1:]:
            combined_df=pd.concat([combined_df, df.loc[df["classification"] == selection]])
    else:
        combined_df = pd.DataFrame()

    return combined_df

def create_laspy_from_dataframe(input_las,combined_df):
    classified_points = laspy.create(point_format=input_las.header.point_format, file_version=input_las.header.version)
    classified_points.X = combined_df["X"]
    classified_points.Y = combined_df["Y"]
    classified_points.Z = combined_df["Z"]
    classified_points.intensity = combined_df["intensity"]
    classified_points.classification = combined_df["classification"]

    return classified_points

def get_numpy_points_from_laspy_scaled(classified_points):
    point_records = classified_points.points.copy()
    # getting scaling and offset parameters
    las_scaleX = classified_points.header.scale[0]
    las_offsetX = classified_points.header.offset[0]
    las_scaleY = classified_points.header.scale[1]
    las_offsetY = classified_points.header.offset[1]
    las_scaleZ = classified_points.header.scale[2]
    las_offsetZ = classified_points.header.offset[2]
    # calculating coordinates
    p_X = np.array((point_records.X * las_scaleX) + las_offsetX) 
    p_Y = np.array((point_records.Y * las_scaleY) + las_offsetY)
    p_Z = np.array((point_records.Z * las_scaleZ) + las_offsetZ)

    classified_points_numpy = np.array(list(zip(p_X,p_Y,p_Z)))

    return classified_points_numpy


def get_numpy_points_from_laspy_unscaled(classified_points):
    point_records = classified_points.points.copy()
    # calculating coordinates
    p_X = np.array(point_records.X)
    p_Y = np.array(point_records.Y)
    p_Z = np.array(point_records.Z)

    classified_points_numpy = np.array(list(zip(p_X,p_Y,p_Z)))

    return classified_points_numpy

def get_numpy_points_from_pandas_unscaled(combined_df):

    classified_points_numpy = np.array(list(zip(np.array(combined_df["X"]),
                                    np.array(combined_df["Y"]),
                                    np.array(combined_df["Z"]))))

    return classified_points_numpy

def get_numpy_points_from_pandas_scaled(combined_df):

    classified_points_numpy = np.array(list(zip(np.array(combined_df["x"]),
                                    np.array(combined_df["y"]),
                                    np.array(combined_df["z"]))))

    return classified_points_numpy

#input numpy array to visualize using open3d
def visualize_las(classified_points,df= pd.DataFrame()):
        
    dataset = classified_points
    geom = o3d.geometry.PointCloud()
    geom.points = o3d.utility.Vector3dVector(dataset)
    #geom.colors = o3d.utility.Vector3dVector(x)
    #print(df)

    if df.empty is False:
        print("entered df")
        #coord = o3d.geometry.TriangleMesh().create_coordinate_frame(size=df.X[0], origin=[df.X.mean(), df.Y.mean(), df.Z.mean()])
        #coord = o3d.geometry.TriangleMesh().create_coordinate_frame(size=0.5, origin=[df.X.mean(),df.Y.mean(),df.Z.mean()])
        coord = o3d.geometry.TriangleMesh().create_coordinate_frame(size=0.5, origin=[0,0,0])
        o3d.visualization.draw_geometries([coord,geom])
        #o3d.visualization.draw_geometries([geom])

    else:
        o3d.visualization.draw_geometries([geom])
        print("did not enter df")
    

def multiple_classifications_to_las(classified_points,class_selections):
    ##write to a .las file
    class_selections.sort()
    sorted_classes_str = "_".join(map(str,class_selections))
    las_file_name = "classifications_" + sorted_classes_str +".las"
    classified_points.write(las_file_name)
    print("Created file: " + sorted_classes_str)




In [8]:
input_las = load_laspy(r'C:\Users\jreye\Documents\GitHub\lidar_processing\python38\esri_lidar_converted\Tile_56_sub.las')
classifications = get_list_classifications(input_las=input_las)
classifications_laspy =  get_classifications_laspy(input_las=input_las)




The classifications contained in this .las dataset are: 
[1, 2, 5, 6, 7, 14, 15]
Creating a python dictionary containing the entire dataset and individual classifications as separate laspy objects.
Saving classification number: 1
Saving classification number: 2
Saving classification number: 5
Saving classification number: 6
Saving classification number: 7
Saving classification number: 14
Saving classification number: 15


In [12]:
input_las.header.scales

array([0.01, 0.01, 0.01])

In [9]:
classifications_laspy

{'original_dataset': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 1233135 points, 11 vlrs)>,
 'classification_1': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 227362 points, 0 vlrs)>,
 'classification_2': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 206534 points, 0 vlrs)>,
 'classification_5': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 726373 points, 0 vlrs)>,
 'classification_6': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 17545 points, 0 vlrs)>,
 'classification_7': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 47966 points, 0 vlrs)>,
 'classification_14': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 7080 points, 0 vlrs)>,
 'classification_15': <LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 275 points, 0 vlrs)>}

In [16]:
input_las

<LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 1233135 points, 11 vlrs)>

In [19]:
point_format = input_las.point_format

print(point_format)

print(point_format.id)

<PointFormat(3, 0 bytes of extra dims)>
3


In [21]:
list(point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'red',
 'green',
 'blue']

In [ ]:
# Convert a .LAS file into a pandas object.
def convert_laspy_pandas_esri(input_las):
    # Convert data into pandas DataFrame
    df = pd.DataFrame({"X":input_las.X,"Y":input_las.Y,"Z":input_las.Z,
      "x":np.array(input_las.x),"y":np.array(input_las.y),"z":np.array(input_las.z),
     'intensity': input_las.intensity,
      'return_number': np.array(input_las.return_number),
      'number_of_returns':np.array(input_las.number_of_returns),
      'scan_direction_flag':np.array(input_las.scan_direction_flag),
      'edge_of_flight_line':np.array(input_las.edge_of_flight_line),
      'classification': input_las.classification,
      'synthetic':np.array(input_las.synthetic),
      'key_point':np.array(input_las.key_point),
      'withheld':np.array(input_las.withheld),
      'scan_angle_rank':np.array(input_las.scan_angle_rank),
      'user_data':input_las.user_data,
      'point_source_id':input_las.point_source_id,
      'gps_time':input_las.gps_time,
      'red':input_las.red,
      'green':input_las.green,
      'blue':input_las.blue,    
      })

    return df

In [24]:
df = convert_laspy_pandas_esri(input_las=input_las)


In [25]:
df

,X,Y,Z,x,y,z,intensity,return_number,number_of_returns,scan_direction_flag,...,synthetic,key_point,withheld,scan_angle_rank,user_data,point_source_id,gps_time,red,green,blue
0,-68066,41108,-83148,6391339.34,1905736.08,1050.52,66,1,1,0,...,0,0,0,8,0,335,6.556441e+07,33536,32256,25856
1,-68289,41016,-83151,6391337.11,1905735.16,1050.49,77,1,1,0,...,0,0,0,8,0,335,6.556441e+07,44544,43008,34560
2,-69385,40977,-83144,6391326.15,1905734.77,1050.56,167,1,1,1,...,0,0,0,8,0,335,6.556441e+07,39424,37376,29440
3,-69188,41059,-83174,6391328.12,1905735.59,1050.26,107,1,1,1,...,0,0,0,8,0,335,6.556441e+07,47872,47616,40960
4,-68991,41141,-83161,6391330.09,1905736.41,1050.39,97,1,1,1,...,0,0,0,8,0,335,6.556441e+07,35584,35072,28672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233130,63939,105140,-95999,6392659.39,1906376.40,922.01,171,1,1,0,...,0,0,0,-8,0,1339,6.556587e+07,12544,15104,9728
1233131,63755,105058,-95877,6392657.55,1906375.58,923.23,123,1,1,0,...,0,0,0,-8,0,1339,6.556587e+07,13568,16640,10752
1233132,63496,104937,-96205,6392654.96,1906374.37,919.95,156,1,1,0,...,0,0,0,-8,0,1339,6.556587e+07,14592,17664,11264
1233133,63483,105029,-95953,6392654.83,1906375.29,922.47,131,1,2,1,...,0,0,0,-8,0,1339,6.556587e+07,16896,20480,13312
